# Image comparison
- Size
- Channels (RGB / grayscale)

## Libraries

### General Purposes

In [1]:
import cv2
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import os
from datetime import datetime
from itertools import cycle
from time import time
from sklearn.metrics import confusion_matrix, classification_report, roc_curve, auc
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

### Tensorflow

In [2]:
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras import layers
from tensorflow.python.keras.utils import to_categorical

C:\Users\leohe\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Functions

In [3]:
def resampling(src, r=2):
    """
    Resamples an image n times using the pyrDown function from opencv
    :param src: np.array - image
    :param n: scalar - times
    :return src: np.array - processed image
    """
    for _ in range(r):
        src = cv2.pyrDown(src)
    return src

def load_folders(filedir, n=2, resample=False, r=1/2, color=False):
    """
    Loads the data from n folders, with(out) resampling
    :param filedir: str - folder with all images
    :param n: scalar - number of folder
    :param resample: bool - flag for resampling
    :param r: scalar - number of resamples
    :param color: bool - read color images
    :return stacked: data - images
    :return label: data - label
    """
    folder_list = [folder for folder in os.listdir(filedir)]
    if color:
        stacked  = np.empty((n * 3000, int(200 / (2 * r)), int(200 / (2 * r)), 3), dtype='uint8')
        color_str = "on BGR"
    else:
        stacked  = np.empty((n * 3000, int(200 / (2 * r)), int(200 / (2 * r))), dtype='uint8')
        color_str = "on Grayscale"
    label = np.empty((n * 3000, 1), 
                     dtype='uint8')
    count = 0
    start = time()
    for folder in folder_list[:n]:
        subfolder = FILEDIR + '\\' + folder
        file_list = [file for file in os.listdir(subfolder)]
        print("Startting folder {} at {}".format(folder, datetime.now()))
        for file in file_list:
            if color:
                img = cv2.imread(subfolder + '\\' + file)
            else:
                img = cv2.imread(subfolder + '\\' + file, 0)
            resample_str = "without resamples"
            if resample and r >= 1:
                resample_str = "with {} resamples".format(r)
                img = resampling(img, int(r))
            stacked[count] = img.astype('int8')
            label[count] = LABEL[folder]
            count += 1
    print(u"There were {} files, collected from {} folders, {}, {}.\nExecution Time: \u2248 {:.2f} [min]"
          .format(count, n, color_str, resample_str, (time() - start) / 60))
    return stacked, label


def plot_confusion_matrix(cm, normalize=False, cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    classes = np.arange(0, cm.shape[0])
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        title = 'Normalized confusion matrix'
    else:
        title = 'Confusion matrix, without normalization'

    plt.figure(figsize=(20, 10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    np.set_printoptions(precision=2)
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()


## Constants

In [4]:
LABEL = {'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9,
         'K':10, 'L':11, 'M':12, 'N':13, 'O':14, 'P':15, 'Q':16, 'R':17, 'S':18, 'T':19,
         'U':20, 'V':21, 'W':22, 'X':23, 'Y':24, 'Z':25, 
         'del':26, 'nothing':27, 'space':28}

FILEDIR = r'C:\Users\leohe\Documents\ITC\HIVE\Challenge\asl_alphabet_train'

In [5]:
# stacked, label = load_folders(FILEDIR, n=4)

In [32]:
stacked = np.load('ASL_x_gray.npy')
label   = np.load('ASL_y_gray.npy')

In [33]:
idx = shuffle(np.arange(len(label)))
# stacked, label = shuffle(stacked, label)
idx_train, idx_test = train_test_split(idx, test_size=0.01)
print(len(idx_train), len(idx_test))

86130 870


In [34]:
inputs = layers.Input(shape=stacked[idx_train].shape[1:])
f = layers.Conv1D(64, (5))(inputs)
f = layers.BatchNormalization()(f)
f = layers.Conv1D(32, (5), activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.MaxPool1D()(f)
f = layers.Conv1D(16, (5), activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.Conv1D( 8, (5), activation='relu')(f)
f = layers.BatchNormalization()(f)
f = layers.MaxPool1D()(f)
f = layers.Flatten()(f)
f = layers.Dense(400, activation='relu')(f)
f = layers.Dense(200, activation='relu')(f)
f = layers.Dense(100, activation='relu')(f)
f = layers.Dense(100, activation='relu')(f)
f = layers.Dense( 50, activation='relu')(f)
outputs = layers.Dense(len(np.unique(label[idx_train])), activation='softmax')(f)
model = Model(inputs=inputs, outputs=outputs)
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200)          0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 196, 64)           64064     
_________________________________________________________________
batch_normalization_5 (Batch (None, 196, 64)           256       
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 192, 32)           10272     
_________________________________________________________________
batch_normalization_6 (Batch (None, 192, 32)           128       
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 96, 32)            0         
_________________________________________________________________
conv1d_7 (Conv1D)            (None, 92, 16)            2576      
__________

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x=stacked[idx_train] / np.max(stacked[idx_train]), 
                    y=to_categorical(label[idx_train], len(np.unique(label[idx_train]))), 
                    batch_size=128, epochs=20,) 
#                     validation_split=0.2)

In [ ]:
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.plot(history.history['acc'])
# plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
# summarize history for loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
# plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
loss, acc = model.evaluate(x=stacked[idx_test] / np.max(stacked[idx_test]), y=to_categorical(label[idx_test], len(np.unique(label[idx_test]))))
print("Test Set\tLoss : {:2f}\t\tAccuracy : {:6.2f}%".format(loss, 100 * acc))
targets = np.argmax(to_categorical(label[idx_test], len(np.unique(label[idx_test]))), axis=-1)
probabilities = model.predict(x=stacked[idx_test] / np.max(stacked[idx_test]))
predictions = np.argmax(probabilities, axis=-1)
cm = confusion_matrix(y_true=targets, y_pred=predictions)
# print(cm)
plot_confusion_matrix(cm)

In [ ]:
# plt.figure()
for letter in LABEL:
    for entry, pred, sample in zip(label[idx_test], predictions, stacked[idx_test]):
        if entry[0] != pred and entry[0] == LABEL[letter]:
            plt.figure()
            plt.imshow(sample, cmap='gray')
            plt.title("True {}, Pred {}".format(list(LABEL.keys())[entry[0]], list(LABEL.keys())[pred]))
            plt.show()
#             print(entry[0], pred)
            break
# plt.show()

In [ ]:
filedir = r'C:/Users/leohe/Documents/ITC/HIVE/Challenge/our_asl_test_set/'
folder_list = [folder for folder in os.listdir(filedir)]
test_stacked  = np.empty((17, 200, 200), dtype='uint8')
test_label = np.empty((17, 1), dtype='uint8')
count = 0
for folder in folder_list:
    subfolder = filedir + '\\' + folder
    file_list = [file for file in os.listdir(subfolder)]
    for file in file_list:
        img = cv2.imread(subfolder + '\\' + file, 0)
        img = resampling(img, 3)
        img = cv2.resize(img, (200, 200))
        test_stacked[count] = img.astype('int8')
        test_label[count] = LABEL[folder]
        count += 1

In [ ]:
loss, acc = model.evaluate(x=test_stacked / np.max(test_stacked), y=to_categorical(test_label, 29))
print("Test Set\tLoss : {:2f}\t\tAccuracy : {:6.2f}%".format(loss, 100 * acc))
targets = np.argmax(to_categorical(test_label, 29), axis=-1)
probabilities = model.predict(x=test_stacked / np.max(test_stacked))
predictions = np.argmax(probabilities, axis=-1)
cm = confusion_matrix(y_true=targets, y_pred=predictions)
# print(cm)
plot_confusion_matrix(cm)

In [ ]:
for pred, sample, prob in zip(predictions, test_stacked, probabilities):
    if pred != 0:
        for index, p in enumerate(prob):
            print("{} : {:6.2f}%".format(index, 100 * p))
        plt.figure()
        plt.imshow(sample, cmap='gray')
        plt.title("{}".format(list(LABEL.keys())[pred]))
        plt.show()        